In [1]:
%load_ext autoreload

In [80]:
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import MySQLConnector, haversine_np
import pandas as pd
import numpy as np
from itertools import combinations

In [51]:
connector = MySQLConnector()
cursor = connector.cursor
db_connection = connector.db_connection

Connected to: 8.1.0
You are connected to the database: ('mysql',)
-----------------------------------------------



### Task 1
How many users, activities and trackpoints are there in the dataset?

In [16]:
query = """
    SELECT 
    (SELECT COUNT(*) FROM UserTable) AS user_count,
    (SELECT COUNT(*) FROM ActivityTable) AS activity_count,
    (SELECT COUNT(*) FROM TrackPointTable) AS trackpoint_count;
"""

cursor.execute(query)
result = cursor.fetchall()
result_df = pd.DataFrame(result, columns=['user_count', 'activity_count', 'trackpoint_count'])
result_df

,user_count,activity_count,trackpoint_count
0,182,14715,3851557


### Task 2
Find the average, maximum and minimum number of trackpoints per user.

In [20]:
query = """
    SELECT
    AVG(count) AS average,
    MAX(count) AS max,
    MIN(count) AS min
    
    FROM 
    (SELECT user_id, COUNT(*) AS count
    FROM TrackPointTable 
    JOIN ActivityTable ON TrackPointTable.activity_id = ActivityTable.id
    GROUP BY user_id) AS TPperUser;
"""

cursor.execute(query)
result = cursor.fetchall()
result_df = pd.DataFrame(result, columns=['average', 'max', 'min'])
result_df

,average,max,min
0,60180.5781,603380,7


### Task 3
Find the top 15 users with the highest number of activities.

In [22]:
query = """
    SELECT user_id, COUNT(*) AS activity_count
    FROM ActivityTable
    GROUP BY user_id
    ORDER BY activity_count DESC
    LIMIT 15; 
"""

cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'number_of_activities'])
result_df

,user_id,number_of_activities
0,163,3182
1,085,1298
2,153,1123
3,068,969
4,167,944
5,128,937
6,062,782
7,075,509
8,126,468
9,010,434


### Task 4
Find all users who have taken a bus.

In [23]:
query = """
    SELECT DISTINCT user_id
    FROM ActivityTable
    WHERE transportation_mode = 'bus';
"""

cursor.execute(query)
result = cursor.fetchall()

print(result)

[('010',), ('020',), ('052',), ('053',), ('058',), ('062',), ('064',), ('065',), ('067',), ('068',), ('069',), ('073',), ('075',), ('078',), ('080',), ('081',), ('082',), ('084',), ('085',), ('091',), ('092',), ('096',), ('098',), ('100',), ('101',), ('102',), ('104',), ('105',), ('108',), ('110',), ('111',), ('112',), ('114',), ('125',), ('126',), ('128',), ('129',), ('138',), ('139',), ('141',), ('147',), ('153',), ('154',), ('161',), ('163',), ('167',), ('174',), ('175',), ('179',)]


### Task 5
List the top 10 users by their amount of different transportation modes.

In [24]:
query = """
    SELECT user_id, COUNT(DISTINCT transportation_mode) AS distinct_transportation_mode_count
    FROM ActivityTable
    GROUP BY user_id
    ORDER BY distinct_transportation_mode_count DESC
    LIMIT 10;
"""

cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'amount_of_different_transportation_modes'])
result_df

,user_id,amount_of_different_transportation_modes
0,128,10
1,075,9
2,062,9
3,163,9
4,167,8
5,153,8
6,084,8
7,126,8
8,020,7
9,085,7


### Task 6
Find activities that are registered multiple times. You should find the query even
if it gives zero result.

In [25]:
query = """
    SELECT user_id, transportation_mode, start_date_time, end_date_time, COUNT(*)
    FROM ActivityTable
    GROUP BY user_id, transportation_mode, start_date_time, end_date_time
    HAVING COUNT(*) > 1;
"""

cursor.execute(query)
result = cursor.fetchall()

result

[]

### Task 7

#### a) 
Find the number of users that have started an activity in one day and ended
the activity the next day.

In [32]:
query = """
    SELECT user_id, transportation_mode, TIMESTAMPDIFF(MINUTE, start_date_time, end_date_time) AS duration_minutes
    FROM ActivityTable
    WHERE DATE(start_date_time) != DATE(end_date_time);
"""

cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'transportation mode', 'duration (min)'])
print("Number of users", result_df["user_id"].unique().shape[0])


#### b)
List the transportation mode, user id and duration for these activities.

In [33]:
result_df

,user_id,transportation mode,duration (min)
0,098,taxi,730
1,106,car,1439
2,153,taxi,556
3,021,car,360
4,076,car,538
...,...,...,...
398,163,car,1079
399,163,car,1079
400,163,car,1019
401,163,car,1019


### Task 8
Find the number of users which have been close to each other in time and space.
Close is defined as the same space (50 meters) and for the same half minute (30
seconds)

In [82]:
query = """
    SELECT id, user_id, start_date_time, end_date_time FROM ActivityTable
"""
cursor.execute(query)
activity_result = cursor.fetchall()
activity_result_df = pd.DataFrame(activity_result, columns=['id', 'user_id', 'start_date_time', 'end_date_time'])

In [84]:
query = """
    SELECT activity_id, lat, lon, altitude, date_time FROM TrackPointTable
"""
cursor.execute(query)
result = cursor.fetchall()

In [86]:
result_df = pd.DataFrame(result, columns=["activity_id", "lat", "lon", "altitude", "date_time"])
result_df["date_time"] = pd.to_datetime(result_df["date_time"]).astype(int) / 10**9

max_date = result_df["date_time"].max()
min_date = result_df["date_time"].min()

In [87]:
from tqdm import tqdm

NUM_BATCHES = 20000
step = (max_date - min_date) / NUM_BATCHES

pbar = tqdm(range(0, NUM_BATCHES))

result = []
for i in pbar:
    tmp = result_df[
        (result_df["date_time"] > i * step + min_date)
        & (result_df["date_time"] < (i + 1) * step + 60 + min_date)
    ]
    pbar.set_postfix({"batch": i, "shape": tmp.shape})

    if tmp.shape[0] == 0:
        continue

    tmp = pd.merge(tmp, activity_result_df, left_on="activity_id", right_on="id")
    tmp["date_time"] = pd.to_datetime(tmp["date_time"] * 10**9)

    user_ids = tmp["user_id"].unique()
    unique_combinations = []
    for combo in combinations(user_ids, 2):
        if combo[::-1] not in unique_combinations:
            unique_combinations.append(combo)

    for combo in unique_combinations:
        if (combo[0], combo[1]) in result or (combo[1], combo[0]) in result:
            continue
        
        tmp_u1 = tmp[tmp["user_id"] == combo[0]]
        tmp_u2 = tmp[(tmp["user_id"] == combo[1])]

        combo_df = tmp_u1.merge(tmp_u2, how="cross")
        filtered_df = combo_df[
            (np.abs(combo_df.altitude_x - combo_df.altitude_y) < 5)
            & (
                abs(combo_df["date_time_x"] - combo_df["date_time_y"])
                <= pd.Timedelta(seconds=30)
            )
        ]

        if filtered_df.shape[0] > 0:
            distances = haversine_np(
                filtered_df["lon_x"],
                filtered_df["lat_x"],
                filtered_df["lon_y"],
                filtered_df["lat_y"],
            ) * 1000

            # Check if any distance is below 5 meters
            has_distance_below_5m = any(distances < 5)

            if has_distance_below_5m:
                result.append((combo[0], combo[1]))

100%|██████████| 20000/20000 [05:10<00:00, 64.42it/s, batch=2e+4, shape=(352, 5)]   


In [27]:
print("UserID combinations:")
result

UserID combinations:


[('111', '128'),
 ('139', '108'),
 ('175', '163'),
 ('175', '056'),
 ('175', '114'),
 ('163', '056'),
 ('174', '101'),
 ('167', '126'),
 ('167', '163'),
 ('163', '089'),
 ('144', '089'),
 ('153', '163'),
 ('128', '153'),
 ('081', '125'),
 ('128', '163'),
 ('167', '129'),
 ('167', '112'),
 ('153', '081'),
 ('153', '125'),
 ('167', '153'),
 ('167', '073'),
 ('153', '073'),
 ('153', '112'),
 ('078', '112'),
 ('163', '112'),
 ('167', '069'),
 ('129', '069'),
 ('073', '112'),
 ('167', '096'),
 ('084', '085'),
 ('078', '073'),
 ('167', '078'),
 ('062', '064'),
 ('128', '179'),
 ('102', '020')]

-------

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [29]:
query = """
    SELECT activity_id, altitude, date_time FROM TrackPointTable
"""
cursor.execute(query)
result = cursor.fetchall()

In [81]:
query = """
    SELECT id, user_id FROM ActivityTable
"""
cursor.execute(query)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id"])

In [82]:
result_df = pd.DataFrame(result, columns=["activity_id", "altitude", "date_time"])

result_df["altitude_t-1"] = result_df.groupby("activity_id")["altitude"].shift(1)
result_df["diff"] = result_df["altitude"] - result_df["altitude_t-1"]
result_df = result_df[~result_df["diff"].isna()]
result_df = result_df[result_df["diff"] > 0]
diff_df = result_df.copy().merge(user_activity_df, on="activity_id")

In [91]:
diff_df.groupby("user_id")["diff"].sum().reset_index().sort_values(
    "diff", ascending=False
).head(15)

,user_id,diff
41,128,873882.59658
19,085,852330.00000
7,062,500368.40000
18,084,435454.00000
52,167,426178.59248
48,153,370663.81548
2,052,268203.00000
40,126,180439.95270
51,163,164278.39590
0,010,163642.00000


### Task 10
Find the users that have traveled the longest total distance in one day for each
transportation mode.

In [4]:
query = """
    SELECT activity_id, lat, lon, date_time FROM TrackPointTable
"""
cursor.execute(query)
track_points = cursor.fetchall()
track_points_df = pd.DataFrame(
    track_points, columns=["activity_id", "lat", "lon", "date_time"]
)

In [21]:
query = """
    SELECT id, user_id, transportation_mode FROM ActivityTable
"""
cursor.execute(query)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id", "transportation_mode"])

In [22]:
user_track_points_df = track_points_df.merge(user_activity_df, on="activity_id")
user_track_points_df

,activity_id,lat,lon,date_time,user_id,transportation_mode
0,1141041,39.9661,116.341,2008-01-01 09:42:31,104,bus
1,1141041,39.9661,116.341,2008-01-01 09:42:34,104,bus
2,1141041,39.9661,116.342,2008-01-01 09:42:37,104,bus
3,1141041,39.9661,116.342,2008-01-01 09:42:40,104,bus
4,1141041,39.9661,116.342,2008-01-01 09:42:43,104,bus
...,...,...,...,...,...,...
3851552,1341471,39.8921,116.329,2011-03-06 11:18:40,147,walk
3851553,1341471,39.8921,116.329,2011-03-06 11:18:45,147,walk
3851554,1341471,39.8921,116.329,2011-03-06 11:18:50,147,walk
3851555,1341471,39.8921,116.329,2011-03-06 11:18:55,147,walk


In [23]:
shifted_points = user_track_points_df.copy()
shifted_points["date"] = pd.to_datetime(shifted_points["date_time"]).dt.date

shifted_points = shifted_points.groupby(["user_id", "activity_id", "date"])[
    ["user_id", "activity_id", "date", "lat", "lon"]
].shift(1)
missing_indices = shifted_points[shifted_points["user_id"].isna()].index
shifted_points = shifted_points.dropna()
shifted_points

,user_id,activity_id,date,lat,lon
1,104,1141041.0,2008-01-01,39.9661,116.341
2,104,1141041.0,2008-01-01,39.9661,116.341
3,104,1141041.0,2008-01-01,39.9661,116.342
4,104,1141041.0,2008-01-01,39.9661,116.342
5,104,1141041.0,2008-01-01,39.9661,116.342
...,...,...,...,...,...
3851552,147,1341471.0,2011-03-06,39.8921,116.329
3851553,147,1341471.0,2011-03-06,39.8921,116.329
3851554,147,1341471.0,2011-03-06,39.8921,116.329
3851555,147,1341471.0,2011-03-06,39.8921,116.329


In [24]:
user_track_points_df = user_track_points_df.drop(missing_indices)
user_track_points_df

,activity_id,lat,lon,date_time,user_id,transportation_mode
1,1141041,39.9661,116.341,2008-01-01 09:42:34,104,bus
2,1141041,39.9661,116.342,2008-01-01 09:42:37,104,bus
3,1141041,39.9661,116.342,2008-01-01 09:42:40,104,bus
4,1141041,39.9661,116.342,2008-01-01 09:42:43,104,bus
5,1141041,39.9661,116.343,2008-01-01 09:42:46,104,bus
...,...,...,...,...,...,...
3851552,1341471,39.8921,116.329,2011-03-06 11:18:40,147,walk
3851553,1341471,39.8921,116.329,2011-03-06 11:18:45,147,walk
3851554,1341471,39.8921,116.329,2011-03-06 11:18:50,147,walk
3851555,1341471,39.8921,116.329,2011-03-06 11:18:55,147,walk


In [37]:
distances = haversine_np(
    user_track_points_df["lon"], user_track_points_df["lat"],
    shifted_points["lon"], shifted_points["lat"],
)

In [48]:
result_df = user_track_points_df.copy()
result_df["date"] = pd.to_datetime(result_df["date_time"]).dt.date
result_df["distance (km)"] = distances
sums = result_df.groupby(["user_id", "date", "transportation_mode"])[["distance (km)"]].sum().sort_values("distance (km)", ascending=False)

In [49]:
sums.head(10)

distance (km)
user_id date       transportation_mode               
128     2009-03-06 airplane               2529.993064
        2008-11-20 airplane               2355.777554
        2009-03-05 airplane               1864.304643
        2009-01-19 airplane               1626.879002
010     2008-03-30 train                  1545.350918
128     2008-09-30 airplane               1443.608643
        2008-11-25 airplane               1345.601040
010     2008-10-03 airplane               1335.887859
052     2008-06-29 airplane               1202.643997
        2008-07-31 airplane               1202.276169

### Task 11
Find all users who have invalid activities, and the number of invalid activities per 
user.

 An invalid activity is defined as an activity with consecutive trackpoints
where the timestamps deviate with at least 5 minutes.

In [53]:
query = """
    SELECT activity_id, date_time FROM TrackPointTable
"""
cursor.execute(query)
track_points = cursor.fetchall()
track_points_df = pd.DataFrame(
    track_points, columns=["activity_id", "date_time"]
)

In [54]:
query = """
    SELECT id, user_id FROM ActivityTable
"""
cursor.execute(query)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id"])

In [61]:
user_track_points_df = track_points_df.merge(user_activity_df, on="activity_id")
user_track_points_df["date_time"] = pd.to_datetime(user_track_points_df["date_time"])

In [64]:
user_track_points_df["date_time_t-1"] = user_track_points_df.groupby("activity_id")[
    "date_time"
].shift(1)
user_track_points_df = user_track_points_df.dropna()

invalid_activities = user_track_points_df[
    (user_track_points_df["date_time"] - user_track_points_df["date_time_t-1"])
    > pd.Timedelta(minutes=5)
][["activity_id", "user_id"]]

In [77]:
invalid_activities.drop_duplicates().groupby("user_id").count().sort_values(
    "activity_id", ascending=False
)

,activity_id
user_id,
128,230
085,208
062,182
084,152
153,145
167,130
068,126
052,106
126,95


### Task 12
Find all users who have registered transportation_mode and their most used
transportation_mode.

In [40]:
connector.close_connection()

NameError: name 'connector' is not defined